In [2]:
#import required modules
import sqlite3 
import numpy as np 
import pandas as pd 
import os
import pyodbc 

#Custom functions found in the sqlite.py file
from sqlite import *

In [2]:
os.getcwd()

'c:\\Users\\khartless\\Documents\\AzureDataStudioExample'

## Create the database

In [3]:
conn = sqlite3.connect('test.db')
c = conn.cursor()

## Import the file into a pandas data frame

In [4]:
dfEggWeight = pd.read_excel('EggWeight.xlsx')
dfEggWeight.head()

,ComplexEntityNo,Weeks,WeekEnd,AddedF,AddedM,HatchEggsProd,CullEggsProd,TotalEggsProd,U_HatchEggProd,U_HEGradeAProd,...,U_CullEggSoftShellProd,U_CDoubleYolkProd,U_CMisshapedEggProd,U_CSmallEggProd,U_UnusedEggsProd,U_CDestroyedProd,U_CommercialProd,U_CCrackedEggProd,U_COtherProd,EggWeight
0,102.P1-961.QFS,0,2019-10-05,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,102.P1-961.QFS,1,2019-10-12,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,102.P1-961.QFS,2,2019-10-19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,102.P1-961.QFS,3,2019-10-26,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,102.P1-961.QFS,4,2019-11-02,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [5]:
dfEggWeight.shape

(29451, 28)

In [11]:
eggWeightCols = list(dfEggWeight.columns)

## Create the functions to work with sqlite tables

In [7]:
#Create the functions to import the data frame into the database
def table_exists(table_name):
    """
    Checks the open database connection and returns true if the table already exists in database
    """
    c.execute("SELECT count(name) FROM sqlite_master WHERE TYPE = 'table' AND name = '{}'".format(table_name))
    if c.fetchone()[0] == 1:
        return True 
    return False

def create_tables(cols, data_types, table_name):
    """
    If the table does not already exist, creates a table with the supplied column names and data types
    """
    if not table_exists(table_name):
        sql_string = "CREATE TABLE {}(".format(table_name)
        for index, col in enumerate(cols):
            if index == 0:
                sql_string += "{} {}".format(col, data_types[index])
            else:
                sql_string += ", {} {}".format(col, data_types[index])
        sql_string += ")"
        c.execute(sql_string)
        return sql_string

def get_data_types(df):
    """
    Determine the data types to be used in the table based on the data types that exist in the data frame
    This function also converts datetime fields to text
    """
    dfModified = df.copy()
    dataTypesList = []
    dataTypes = dict(df.dtypes)
    dataTypes['ComplexEntityNo']
    for key, value in dataTypes.items():
        if value in ['object', 'datetime64[ns]', '<M8[ns]']:
            dataTypesList.append('TEXT')
            dfModified[key] = [str(x) for x in dfModified[key]]
        elif value in ['int64']:
            dataTypesList.append('INTEGER')
        elif value in ['float64']:
            dataTypesList.append('REAL')
        else:
            dataTypesList.append('BLOB')
    return dfModified, dataTypesList

def insert_data(table_name, cols, df):
    """
    Read each row of the data frame an insert it into the sqlite table
    """
    sql_string = "INSERT INTO {} (".format(table_name)
    for index, col in enumerate(cols):
        if index == 0:
            sql_string += col 
        else:
            sql_string += ", {}".format(col)
    sql_string += ") VALUES("
    sql_string += "?, "*(len(cols)-1)
    sql_string += "?)"
    for index, row in df.iterrows():
        c.execute(sql_string, tuple(row))
        conn.commit()

def to_sqlite(df, table_name, **cols):
    """
    Create a table and populate it based on an existing dataframe
    """
    if (len(cols) == 0) | (len(cols) != len(list(df.columns))) | (type(cols) != 'list'):
        cols = list(df.columns)
    df, dataTypes = get_data_types(df)
    create_tables(cols, dataTypes, table_name)
    insert_data(table_name, cols, df)

def query_table(table_name, top_n):
    if table_exists(table_name):
        if top_n > 0:
            c.execute("SELECT * FROM {} LIMIT {}".format(table_name, top_n))
        else:
            c.execute("SELECT * FROM {}".format(table_name))
        data = []
        for row in c.fetchall():
            data.append(row)
        return data
    else:
        return "Table {} does not exist.".format(table_name)

def delete_table(table_name):
    c.execute("DROP TABLE IF EXISTS {}".format(table_name))


## Import the data frame into the database

In [8]:
to_sqlite(dfEggWeight.head(1000), 'EggWeight')

## Query the data in the database

In [10]:
print(query_table('EggWeight', 5))

[('102.P1-961.QFS', 0, '2019-10-05 00:00:00', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None), ('102.P1-961.QFS', 1, '2019-10-12 00:00:00', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None), ('102.P1-961.QFS', 2, '2019-10-19 00:00:00', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None), ('102.P1-961.QFS', 3, '2019-10-26 00:00:00', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None), ('102.P1-961.QFS', 4, '2019-11-02 00:00:00', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None)]


## Store the query results back into a data frame

In [14]:
dfNew = pd.DataFrame(query_table('EggWeight', 5), columns = eggWeightCols)
dfNew

,ComplexEntityNo,Weeks,WeekEnd,AddedF,AddedM,HatchEggsProd,CullEggsProd,TotalEggsProd,U_HatchEggProd,U_HEGradeAProd,...,U_CullEggSoftShellProd,U_CDoubleYolkProd,U_CMisshapedEggProd,U_CSmallEggProd,U_UnusedEggsProd,U_CDestroyedProd,U_CommercialProd,U_CCrackedEggProd,U_COtherProd,EggWeight
0,102.P1-961.QFS,0,2019-10-05 00:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
1,102.P1-961.QFS,1,2019-10-12 00:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
2,102.P1-961.QFS,2,2019-10-19 00:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
3,102.P1-961.QFS,3,2019-10-26 00:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None
4,102.P1-961.QFS,4,2019-11-02 00:00:00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,None


## Close the connection when done

In [15]:

c.close()

try:
    conn.close()
    print("connection closed")
except:
    print("connection already closed")

connection closed


## Create a connection to a sql database

In [29]:
cs=('Driver={SQL Server};'
   "Server=sp2019db\\aviagen_globaldb;"
   'Database=MtechDataWarehouse;'
   'Trusted_Connection=yes;'
   )

conn = pyodbc.connect(cs)
query = "SELECT TOP 25 ComplexEntityNo FROM vwWeeklyMTechKPI"

dfSqlTest = pd.read_sql(query, conn)


In [30]:
dfSqlTest

,ComplexEntityNo
0,850-UAS1911
1,372-3081987
2,409-YPM1782
3,UKGR-HW1890.MJY
4,FRFO-HW1822.308
5,HFY2-HW1995.F72
6,384-YPM1933
7,HFS1-HW1914.A7
8,732-892UAY
9,718-835QFY
